# Converting Local Data to NWB

This notebook steps through converting data to NWB format. 

Note that this notebook doesn't use the metadata files.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
from pathlib import Path

from datetime import datetime
from dateutil.tz import tzlocal

import h5py
import joblib
import numpy as np
import pandas as pd

from pynwb import NWBFile, NWBHDF5IO, TimeSeries, ProcessingModule
from pynwb.file import Subject
from pynwb.behavior import Position, SpatialSeries
from pynwb.ecephys import ElectricalSeries, SpikeEventSeries

In [ ]:
# Import local code module
import sys
sys.path.append('..')
from conv.utils import clean_strings, get_event_time
from conv.io import get_files, load_task_object

### Settings

In [ ]:
# Define subject information
subj = 'example'
session = 'session_0'

In [ ]:
# Define the base data folder
data_folder = Path('...')

# Define the full subject & session path
full_path = data_folder / subj / session

In [ ]:
# Load behavior data
task = load_task_object(full_path / 'behavioral' / 'task.p')

In [ ]:
# Get current date
current_date = datetime.now(tzlocal())

## Initialize a NWB File

Set up the file.

### Define Subject Information

In [ ]:
# Set subject information
age = None
sex = None
description = 'participant'
species = 'human'
subject_id = 'test'

In [ ]:
# Create subject object
subject = Subject(age=age,
                  sex=sex,
                  description=description, 
                  species=species,
                  subject_id=subject_id)

### Define Recording Metadata

In [ ]:
# Define metadata for NWB file
session_description = 'XX'
identifier = 'XX'
experimenter = ['Experimenter1', 'Experimenter2']
experiment_description = 'Experiment information - example.'
session_id = 'testfile'
institution = 'Columbia'
keywords = ['keyword1', 'keyword2']
source_script='https://github.com/JacobsSU/AnalyzeTH/'
source_script_file_name='01-ConvertToNWB.ipynb'
stimulus_notes = 'Notes on stimulus presentation.'
lab = 'Jacobs Lab'

In [ ]:
# Define information collected from task object
experiment_description = \
    'Task: ' + task.experiment['version']['label'] + \
    ' build-' + task.experiment['version']['number'] + \
    ' ({})'.format(task.experiment['language'])

In [ ]:
# Define collection site information
if subj[0:2] == 'XX':
    data_collection = 'XX'
else:
    data_collection = 'unknown'

### Collect together into NWB file

In [ ]:
# Initialize a NWB file
nwbfile = NWBFile(session_description=session_description,
                  identifier=session_name,
                  session_start_time=session_date,
                  file_create_date=current_date,
                  experimenter=experimenter,
                  experiment_description=experiment_description,
                  session_id=session_id,
                  institution=institution,
                  keywords=keywords,
                  notes=notes,
                  source_script=source_script,
                  source_script_file_name=source_script_file_name,
                  data_collection=data_collection,
                  stimulus_notes=stimulus_notes,
                  lab=lab,
                  subject=subject)

## Recording Definition

### Device(s)

In [ ]:
# Device information
device_name = 'RECORDING DEVICE'
device_desc = 'RECORDING DEVICE DESCRIPTION'
device_manu = 'RECORDING DEVICE MANUFACTURER'

# Create device object
device = nwbfile.create_device(device_name, device_desc, device_manu)

In [ ]:
# Check out the defined device
device

### Electrodes

In [ ]:
# Add electrode description
location = 'WHERE'
electrode_name = '{}-microwires-{}'.format('A', 'chnum')
description = "Behnke Fried/Micro Inner Wire Bundle....ADD DETAILS."

# Add electrode group
electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

In [ ]:
# Define / get electrode information
x_pos, y_pos, z_pos = 0.0, 0.0, 0.0
imp = np.nan
location = 'place'
filtering = '0, np.inf'
reference = None

In [ ]:
# Add electrode to NWB
n_electrodes = 8
for ind in range(n_electrodes):
    nwbfile.add_electrode(x_pos, y_pos, z_pos, imp, location, filtering, electrode_group, 
                          id=ind, reference=reference)

## Stimuli

Add stimuli of interest to the NWB file.

In [ ]:
# Add stimuli
stim_description = 'DESCRIPTION.'
for stim in stimuli:
    nwbfile.add_stimulus(stim)

## Behaviour data

### Trial Data

In [ ]:
# Define trial information
nwbfile.add_trial_column('...', '...')

In [ ]:
# Collect trial indices
trial_inds = ...

In [ ]:
# Add event information to NWB file
n_trials = len(trial_inds)
for t_ind in range(len(task.trial['trial'])):
    
    # Add trial information to file
    try:
        nwbfile.add_trial(start_time=...,
                          ...,
                          stop_time=...
                         )
    except IndexError:
        print('Incomplete last trial - skipped adding.')

In [ ]:
# Check the trial information as a dataframe
behav = nwbfile.trials.to_dataframe()
behav.head()

### Position Data

In [ ]:
# Set position data as a spatial series and add to NWB file
position = Position(name='position')
position.create_spatial_series(name='player_position',
                               data=np.vstack([task.pos['x'], task.pos['z']]),
                               unit='virtual units',
                               timestamps=task.position['time'],
                               reference_frame='XX',
                               description='Position of the subject.')
nwbfile.add_acquisition(position)

#### Add position derived measures as ProcessingModule

In [ ]:
# Create time series for speed & linear positon
speed = TimeSeries(name='speed',
                   description='The players movement speed, computed from the position data.',
                   data = task.position['speed'],
                   unit = 'virtual units / second',
                   timestamps=task.position['time'])

In [ ]:
# Add derived spatial measures to NWB file as ProcessingModule
position_things = ProcessingModule(name='position_measures',
                                   description='Derived measures related to position data.',
                                   data_interfaces=[speed])
nwbfile.add_processing_module(position_things)

## Spiking Data

In [ ]:
# Get a list of the available spike files
spike_files = get_files(full_path / 'spikes')

In [ ]:
# Specify additional metadata columns for units
nwbfile.add_unit_column('channel', 'The recording channel of this unit.')
nwbfile.add_unit_column('location', 'The anatomical location of this unit.')

In [ ]:
# Add each unit to the NWB file
for ind, spike_file in enumerate(spike_files):
    
    # Load spike file & get spike data
    # NOTE: currently loads HDF5 file - update as needed
    with h5py.File(full_path / 'spikes' / spike_file, 'r') as h5file:
        spike_data = h5file['spike_data_sorted']
    
        # Add unit data
        nwbfile.add_unit(id=ind,
                         electrodes=[0],
                         channel=...,
                         location=...
                         spike_times=...)

## Field Data

In [ ]:
# Create the electrode table
electrode_table_region = nwbfile.create_electrode_table_region([0], 'xx')

In [ ]:
# Get the list of available LFP files
lfp_files = get_files(full_path / 'lfp', select='.p')

In [ ]:
# Add each LFP trace as a new object
for ind, lfp_file in enumerate(lfp_files):
    with open(full_path / 'lfp' / lfp_file, 'rb') as pfile:
        
        # Load ephys data
        #ephys_data = load(...)
        
        # Create & add electrical series to store LFP data
        ephys_ts = ElectricalSeries('field_data_' + str(ind),
                                    ephys_data,
                                    electrode_table_region,
                                    starting_time=0.,
                                    rate=500.,
                                    resolution=np.inf,
                                    comments="...",
                                    description="LFP time series.")
        nwbfile.add_acquisition(ephys_ts)

### Save out local data file

In [ ]:
# Save out an example NWB file
with NWBHDF5IO('nwb_local_data.nwb', 'w') as io:
    io.write(nwbfile)